This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/trajopt.html).  I recommend having both windows open, side-by-side!

# Trajectory optimization for the double integrator

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from pydrake.all import (
    DirectCollocation,
    DirectTranscription,
    LinearSystem,
    MathematicalProgram,
    Solve,
    eq,
)

# Direct Transcription (using Mathematical Program directly)

In [ ]:
def double_integrator():
    # Discrete-time approximation of the double integrator.
    dt = 0.01
    A = np.eye(2) + dt * np.mat("0 1; 0 0")
    B = dt * np.mat("0; 1")

    def solve_for_fixed_horizon(N):
        prog = MathematicalProgram()

        # Create decision variables
        u = prog.NewContinuousVariables(1, N - 1, "u")
        x = prog.NewContinuousVariables(2, N, "x")

        # Add constraints
        x0 = [-2, 0]
        prog.AddBoundingBoxConstraint(x0, x0, x[:, 0])
        for n in range(N - 1):
            # Will eventually be prog.AddConstraint(x[:,n+1] == A@x[:,n] + B@u[:,n])
            # See drake issues 12841 and 8315
            prog.AddConstraint(
                eq(x[:, n + 1], A.dot(x[:, n]) + B.dot(u[:, n]))
            )
            prog.AddBoundingBoxConstraint(-1, 1, u[:, n])
            prog.AddQuadraticCost(u[0, n] ** 2, True)
        xf = [0, 0]
        prog.AddBoundingBoxConstraint(xf, xf, x[:, N - 1])

        result = Solve(prog)
        return result, prog, x, u

    # TODO(russt): Do a line search here (I've done it manually).
    N = 284
    # True solution: min time(q=-2, qdot=0) = 2√2 = 2.828 seconds.
    result, prog, x, u = solve_for_fixed_horizon(N)
    assert result.is_success(), "Optimization failed"

    u_sol = result.GetSolution(u)
    x_sol = result.GetSolution(x)

    fig, ax = plt.subplots(2, 1)
    ax[0].plot(x_sol[0, :], x_sol[1, :], "-")
    ax[0].set_xlabel("q")
    ax[0].set_ylabel("qdot")

    ax[1].plot(np.arange(0, N - 1) * dt, u_sol.T, "-")
    ax[1].set_xlabel("t")
    ax[1].set_ylabel("u")


double_integrator()

## DirectTranscription (using the DirectTranscription class)

Because this pattern of making decision variables that are indexed over time, adding the dynamic constraints, defining the running cost and constraints, is so common, we have wrappers in drake on top of `MathematicalProgram` which handle these details for you.

The optimization below is identical to the example above, but using this helper class.

In [ ]:
def dirtran_example():
    # Discrete-time approximation of the double integrator.
    dt = 0.01
    A = np.eye(2) + dt * np.mat("0 1; 0 0")
    B = dt * np.mat("0; 1")
    C = np.eye(2)
    D = np.zeros((2, 1))
    sys = LinearSystem(A, B, C, D, dt)

    N = 284
    x0 = [-2, 0]
    xf = [0, 0]

    dirtran = DirectTranscription(sys, sys.CreateDefaultContext(), N)
    prog = dirtran.prog()
    prog.AddBoundingBoxConstraint(x0, x0, dirtran.initial_state())
    prog.AddBoundingBoxConstraint(xf, xf, dirtran.final_state())
    dirtran.AddConstraintToAllKnotPoints(dirtran.input()[0] <= 1)
    dirtran.AddConstraintToAllKnotPoints(dirtran.input()[0] >= -1)
    dirtran.AddRunningCost(dirtran.input()[0] ** 2)

    result = Solve(prog)
    assert result.is_success(), "Optimization failed"
    u_sol = dirtran.ReconstructInputTrajectory(result)
    x_sol = dirtran.ReconstructStateTrajectory(result)

    fig, ax = plt.subplots(2, 1)
    x_values = x_sol.vector_values(x_sol.get_segment_times())
    ax[0].plot(x_values[0, :], x_values[1, :], "-")
    ax[0].set_xlabel("q")
    ax[0].set_ylabel("qdot")

    u_values = u_sol.vector_values(u_sol.get_segment_times())
    ax[1].plot(u_sol.get_segment_times(), u_values.T, "-")
    ax[1].set_xlabel("t")
    ax[1].set_ylabel("u")


dirtran_example()

One thing that I'm very proud of (it was a lot of work!) is the fact that drake is often smart enough to introspect your system, costs, and constraints and understand whether you have formulated a convex problem or a non-convex one.  The optimization above calls a convex optimization solver.  But if you had passed in a nonlinear system instead, it would have switched to calling a solver that supports nonlinear programming.

## Direct Collocation

With only a minor change, we can use `DirectCollocation` instead of `DirectTranscription`.  This works directly on the continuous-time equations, but even though the system is linear, it gives a nonconvex optimization (make sure you understand why!). Satisfyingly, though, this finds the optimal solution (the true minimum time) without needing a line search!

In [ ]:
def dircol_example():
    # Continuous-time double integrator
    A = np.mat("0 1; 0 0")
    B = np.mat("0; 1")
    C = np.eye(2)
    D = np.zeros((2, 1))
    sys = LinearSystem(A, B, C, D)

    N = 41
    x0 = [-2, 0]
    xf = [0, 0]

    dircol = DirectCollocation(
        sys, sys.CreateDefaultContext(), N, 0.001, 10 / N
    )
    prog = dircol.prog()
    prog.AddBoundingBoxConstraint(x0, x0, dircol.initial_state())
    prog.AddBoundingBoxConstraint(xf, xf, dircol.final_state())
    dircol.AddConstraintToAllKnotPoints(dircol.input()[0] <= 1)
    dircol.AddConstraintToAllKnotPoints(dircol.input()[0] >= -1)
    dircol.AddEqualTimeIntervalsConstraints()
    dircol.AddFinalCost(dircol.time())

    result = Solve(prog)
    assert result.is_success(), "Optimization failed"
    u_sol = dircol.ReconstructInputTrajectory(result)
    x_sol = dircol.ReconstructStateTrajectory(result)
    print(f"minimum time = {dircol.GetSampleTimes(result)[-1]}")

    fig, ax = plt.subplots(2, 1)
    x_values = x_sol.vector_values(x_sol.get_segment_times())
    ax[0].plot(x_values[0, :], x_values[1, :], ".-")
    ax[0].set_xlabel("q")
    ax[0].set_ylabel("qdot")

    u_values = u_sol.vector_values(u_sol.get_segment_times())
    ax[1].plot(u_sol.get_segment_times(), u_values.T, ".-")
    ax[1].set_xlabel("t")
    ax[1].set_ylabel("u")


dircol_example()